In [1]:
import numpy as np
from pandas_datareader import data as pdr
import datetime as dt
import mplfinance as mpf
import yfinance as yf
import pandas as pd

In [2]:
start = dt.datetime(2020,1,3)
end = dt.datetime.today()
yf.pdr_override()
df = pdr.get_data_yahoo('HGINFRA.NS',start,end)
df=df[df['Volume']!=0]
df.reset_index(drop=True, inplace=True)
df.isna().sum()
# df.drop('Close',axis='columns',inplace=True)
df.tail()


[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
1072,1199.949951,1200.800049,1151.099976,1163.300049,1163.300049,107825
1073,1163.750000,1169.849976,1117.199951,1125.099976,1125.099976,160000
1074,1122.949951,1195.000000,1106.000000,1185.550049,1185.550049,231454
1075,1219.949951,1227.000000,1125.050049,1141.800049,1141.800049,728429
1076,1147.800049,1187.250000,1141.050049,1181.099976,1181.099976,273013


In [3]:
import pandas_ta as pa
# help(pa.macd)
df.rename(columns = {'Open':'open','High':'high','Low':'low','Close':'close','Volume':'volume'}, inplace =True)

In [4]:
df["RSI"] = pa.rsi(df.close, length=16)
df["CCI"] = pa.cci(df.high, df.low, df.close, length=16)
df["AO"] = pa.ao(df.high, df.low)
df["MOM"] = pa.mom(df.close, length=16)
a = pa.macd(df.close)
df = df.join(a)
df["ATR"] = pa.atr(df.high, df.low, df.close, length=16)
df["BOP"] = pa.bop(df.open, df.high, df.low, df.close, length=16)
df["RVI"] = pa.rvi(df.close)
a = pa.dm(df.high, df.low, length=16)
df = df.join(a)
a = pa.stoch(df.high, df.low, df.close)
df = df.join(a)
a = pa.stochrsi(df.close, length=16)
df = df.join(a)
df["WPR"] = pa.willr(df.high, df.low, df.close, length=16)

In [5]:
df['5daysaftercp']=df.close
df['5daysaftercp'] =df['5daysaftercp'].shift(-3)
df['Target'] = df['5daysaftercp']-df.open

In [6]:
# df['y'] = [1 if (df.Target[i]>0 and (df.Target[i]/df.close[i])>0.05) else 0 for i in range(len(df))]
# df['y'] = [1 if (df.Target[i]>0 and (df.Target[i]/df.close[i])>0.05) else 0 for i in range(len(df))]
df['y']=0
for i in range(len(df)):
    if(df.Target[i]>0 and (df.Target[i]/df.close[i])>0.05):
        df.y[i]=1
    elif(df.Target[i]<0 and (abs(df.Target[i])/df.close[i])>0.05):
        df.y[i]=0
    else:
        df.y[i]=2
    

C:\Users\91907\AppData\Local\Temp\ipykernel_30456\1299771411.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.y[i]=2
C:\Users\91907\AppData\Local\Temp\ipykernel_30456\1299771411.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.y[i]=2
C:\Users\91907\AppData\Local\Temp\ipykernel_30456\1299771411.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.y[i]=2
C:\Users\91907\AppData\Local\Temp\ipykernel_30456\1299771411.py:10:

In [7]:
df.isna().sum()
df.dropna(inplace=True)
df.isna().sum()

open                   0
high                   0
low                    0
close                  0
Adj Close              0
volume                 0
RSI                    0
CCI                    0
AO                     0
MOM                    0
MACD_12_26_9           0
MACDh_12_26_9          0
MACDs_12_26_9          0
ATR                    0
BOP                    0
RVI                    0
DMP_16                 0
DMN_16                 0
STOCHk_14_3_3          0
STOCHd_14_3_3          0
STOCHRSIk_16_14_3_3    0
STOCHRSId_16_14_3_3    0
WPR                    0
5daysaftercp           0
Target                 0
y                      0
dtype: int64

In [8]:
df.reset_index(drop=True,inplace=True)
df.head()
# df.tail()

,open,high,low,close,Adj Close,volume,RSI,CCI,AO,MOM,...,DMP_16,DMN_16,STOCHk_14_3_3,STOCHd_14_3_3,STOCHRSIk_16_14_3_3,STOCHRSId_16_14_3_3,WPR,5daysaftercp,Target,y
0,242.000000,249.899994,242.00,246.100006,245.002914,15416,41.628641,-100.747698,-14.116766,-20.850006,...,1.401622,2.612854,36.976938,38.611554,43.910223,36.857324,-61.076335,244.949997,2.949997,2
1,246.000000,255.899994,244.00,253.199997,252.071243,17544,46.645260,-28.928758,-14.768091,-13.449997,...,1.725063,2.429071,40.261933,37.917788,61.873623,46.427824,-42.585562,233.449997,-12.550003,2
2,252.000000,252.000000,246.75,248.350006,247.242874,115200,43.896403,-52.387457,-15.167943,-5.000000,...,1.604669,2.259543,47.317283,41.518718,74.735966,60.173270,-54.879583,238.800003,-13.199997,0
3,248.949997,250.949997,242.00,244.949997,243.858002,31926,42.043669,-96.949268,-15.512796,-22.500015,...,1.493488,2.432097,40.179375,42.586197,73.744379,70.117989,-63.498112,240.100006,-8.849991,2
4,236.000000,241.699997,231.00,233.449997,232.409286,22127,36.487508,-212.741413,-16.901324,-11.850006,...,1.390708,3.021731,23.135244,36.877301,50.146241,66.208862,-92.648938,229.100006,-6.899994,2


In [9]:
# train_length = int(0.80*len(df))
attributes = ['Adj Close','RSI', 'CCI', 'AO', 'MOM', 'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'ATR',
       'BOP', 'RVI', 'DMP_16', 'DMN_16', 'STOCHk_14_3_3', 'STOCHd_14_3_3',
       'STOCHRSIk_16_14_3_3', 'STOCHRSId_16_14_3_3', 'WPR']

X=df[attributes]
y=df['y']
print(y)
# print(X), print(y)
X_test=X[:]
y_test=y[:]

0       2
1       2
2       0
3       2
4       2
       ..
1036    2
1037    0
1038    2
1039    2
1040    2
Name: y, Length: 1041, dtype: int64


In [10]:
import joblib
filename='fivedaycall(titan)_n.sav'
classifier = joblib.load(open(filename, 'rb'))

In [ ]:
from sklearn.metrics import accuracy_score, log_loss,confusion_matrix
# train_pred = classifier.predict(X_train)
test_pred = classifier.predict(X_test)
# acc_train = accuracy_score(y_train, train_pred)
acc_test = accuracy_score(y_test, test_pred)
# print('****Train Results****')
# print("Accuracy: {:.4%}".format(acc_train))
print('****Test Results****')
print("Accuracy: {:.4%}".format(acc_test))